In [ ]:
%%html
<script type=text/javascript>
/* Add a button for showing or hiding input */
on = "Show input";
off = "Hide input";
function onoff(){
  currentvalue = document.getElementById('onoff').value;
  if(currentvalue == off){
    document.getElementById("onoff").value=on;
      $('div.input').hide();
  }else{
    document.getElementById("onoff").value=off;
      $('div.input').show();
  }
}

/* Launch first notebook cell on start */
function launch_first_cell (evt) {
  if (!launch_first_cell.executed
      && Jupyter.notebook.kernel
  ) {
    Jupyter.notebook.get_cells()[0].execute();
    launch_first_cell.executed = true;
  }
}
$(document).ready(function (){onoff();})
</script>

<p>Press this button to show/hide the code used in the notebook:
<input type="button" class="ui-button ui-widget ui-state-default \
ui-corner-all ui-button-text-only" value="Hide input" id="onoff" \
onclick="onoff();"></p>


# Data processing, skip further for discussion.

In [ ]:
%matplotlib inline

import os
from collections import OrderedDict, Counter

from matplotlib import pyplot
import pandas
from pony import orm
from IPython import display
import jinja2
import db
exam_id = 

In [ ]:
db.use_db('course.sqlite')

In [ ]:
data = {}

with orm.db_session:
    exam = db.Exam[exam_id]
    problem_names = [p.name for p in exam.problems.order_by(db.Problem.id)
                     if orm.count(p.solutions.feedback)]
    possible_feedback = OrderedDict()
    for problem in exam.problems.order_by(db.Problem.id):
        if not orm.count(problem.solutions.feedback):
            continue
        for fb in problem.feedback_options:
            possible_feedback[problem.name + ': ' + fb.text] = (problem.id, fb.score, fb.description)

    for sub in exam.submissions:
        a = []
        for sol in sub.solutions:
            a.extend([sol.problem.name + ': ' + fb.text for fb in sol.feedback])

        data[sub.student.id] = [
            option in a for option in possible_feedback
        ]
        
data = pandas.DataFrame(data, index=possible_feedback).T
grading_scheme = pandas.DataFrame(possible_feedback, index=['Problem number', 'Score', 'Desciption'])
grading_scheme.T['Score'][grading_scheme.T['Score'].isnull()] = 0

points = (data * grading_scheme.T['Score'])
points = points.astype(int)
problem_scores = points.T.groupby(grading_scheme.T['Problem number']).sum()
problem_scores = problem_scores.applymap(lambda x: max(0, x))
problem_scores.index = problem_names
problem_scores.ix['Total'] = problem_scores.sum(axis=0)

# Points per problem and the total score

In [ ]:
for index, name in zip(problem_scores.T, problem_scores.index):
    pyplot.figure()
    problem_scores.T[index].hist(bins=20)
    pyplot.title(name)
    pyplot.xlabel = 'Score'
    pyplot.ylabel = 'Number'

# More summary statistics

In [ ]:
problem_scores.T.describe()

# Correlations between different problems.

In [ ]:
problem_scores.T.corr().round(2)

# Lenghty description of all possible feedback options and how often they were received

In [ ]:
feedback_template = jinja2.Template("""<ul>
{% for fo in feedback_options %}
<li> {{ fo.text }}: {{ fo.description }} </li>
{% endfor %}
</ul>
""")

stats = ''

for problem in db.Exam[exam_id].problems.order_by(lambda p: p.name):
    if not orm.max(problem.feedback_options.score, default=0):
        continue
    df = pandas.DataFrame({fo.text: (fo.solutions.count(), fo.score) 
                           for fo in problem.feedback_options if fo.solutions.count()}, 
                          index=['amount', 'score']).T.fillna(0).astype(int)
    df.index.name = "Feedback"
    stats += '<h2>' + problem.name + '</h2>'
    stats += '<h4>Feedback frequencies and scores</h4>'
    stats += df._repr_html_()
    df = pandas.DataFrame(pandas.Series(Counter(sum(fo.score or 0 for fo in sol.feedback) 
                          for sol in problem.solutions)), columns=['Amount'])
    df.index.name = 'Score'
    stats += '<hr><h4>Score distribution</h4>'
    stats += df.T._repr_html_()
    stats += '<hr><h4>Descriptions</h4>'
    stats += feedback_template.render(feedback_options=(fo for fo in problem.feedback_options
                                                        if fo.solutions.count()))


stats = display.HTML(stats)
stats